In [8]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks.callbacks import LearningRateScheduler

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_batches = test_datagen.flow_from_directory('data/5_class/val',
                                                  target_size=(224,224),
                                                  class_mode='categorical', shuffle=False,
                                                  batch_size=4)
# show class indices
print('****************')
for cls, idx in test_batches.class_indices.items():
    print('Class nr ',idx,' -> ', cls)
print('****************')


net = MobileNetV2(include_top=False,
                        weights='imagenet',
                        input_tensor=None,
                        input_shape=(224,224,3))
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(5, activation='softmax', name='softmax')(x)
loaded_model = Model(inputs=net.input, outputs=output_layer)


# load weights into new model
loaded_model.load_weights("models/AM_mobilenetV2.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(optimizer=Adam(lr=5e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

score = loaded_model.evaluate(test_batches, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Found 4565 images belonging to 5 classes.
****************
Class nr  0  ->  Acroc
Class nr  1  ->  Ember
Class nr  2  ->  Parus
Class nr  3  ->  Phyll
Class nr  4  ->  Sylvi
****************
Loaded model from disk
acc: 80.37%


In [13]:
from keras import metrics

Y_pred = loaded_model.predict_generator(test_batches, 4565 // 4+1)

NameError: name 'np' is not defined

In [19]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_batches.classes, y_pred))
print('Classification Report')
target_names = ['Acroc', 'Ember', 'Parus','Phyll','Sylvi']
print(classification_report(test_batches.classes, y_pred, target_names=target_names))


Confusion Matrix
[[ 367    8   14    1   22]
 [   4  499   96   51  187]
 [   0   41  570   33  122]
 [   0   20   22  347   82]
 [   1   67   68   57 1886]]
Classification Report
              precision    recall  f1-score   support

       Acroc       0.99      0.89      0.94       412
       Ember       0.79      0.60      0.68       837
       Parus       0.74      0.74      0.74       766
       Phyll       0.71      0.74      0.72       471
       Sylvi       0.82      0.91      0.86      2079

    accuracy                           0.80      4565
   macro avg       0.81      0.77      0.79      4565
weighted avg       0.80      0.80      0.80      4565

